## 期末專題題目:
### 議會質詢統計

- 1.類別分析
    - 哪類較多筆
    - 個人各類幾筆
- 2.個別次數分析
    - 個人總共幾次書面質詢
    - 某個時間區間此類別很多筆
- 3.時間分析
    - 某議員在特定月份較多書面質詢




In [1]:
import re
import numpy as np
import pandas as pd
import requests as rq
from bs4 import BeautifulSoup
import matplotlib.pyplot as plt 
import matplotlib as mpl

In [2]:
table = []
for i in range(29,108):  # 29~107
    url = f"https://tcckm.tcc.gov.tw/tccgazFront/qnews/qnews_forTCC.jsp?t=4&groupingType=1&gTrmPrdCnltype=&dept_name=&page={i}"
    try:
        response = rq.get(url)              # 用 requests 的 get 方法把網頁抓下來
        # print("下載成功")
    except Exception as err:
        print(f"網頁下載失敗: {err}")
        break
    
    html_doc = response.text                    # text 屬性就是 html 檔案(全部)
    soup = BeautifulSoup(html_doc, "lxml")      # 指定 lxml 作為解析器(才能搜尋想爬的內容)
    fs = soup.find_all("td", class_="center")   # 爬出(書面質詢紀錄)所有內容
    
    all_txt = []                                # 只保留 text 並以 list 存起來(每一頁用list存起來才能用split&slice)
    for f in fs:
        all_txt.append(f.text)
    
    
    data = all_txt[0].split('\n')                           # all_txt 以換行符號分開
    new_data = data[178:-8]                                 # 保留需要的資訊
    data_list = [x for x in new_data if not x.isdigit()]    # 扣除順序(以利顯示)

    page_txt = [t for t in data_list if len(t) > 1]                         # 扣除''(只保留文字)
    page_txt = [re.sub(r'。','',t)for t in page_txt]                        # 扣除句號(後來發現有句號在名字裡)
    page_txt = page_txt[6:]
    page_detail = [page_txt[i:i+4] for i in range(0, len(page_txt), 4)]     # 每6個元素一組建立新的 list
    table.append(page_detail)

table = np.concatenate(table, axis=0)       # 因為不同頁面list會分隔,我只要一個[]就好       
table = np.asarray(table)                   # 存成 numpy array
table = np.delete(table,2,1)                # 去除質詢題目(字太多會爬很久)
print(table)                                # 確認內容


網頁下載失敗: ('Connection aborted.', RemoteDisconnected('Remote end closed connection without response'))


ValueError: need at least one array to concatenate

In [25]:
df = pd.DataFrame(table, columns=["Date", "Category", "Name"])     # 用 pandas dataframe 輸出
df.to_csv(f"./table_29to107.csv", encoding="utf_8_sig")
df

,Date,Category,Name
0,民國113年02月06日,教育,張志豪
1,民國113年02月06日,財政建設,張志豪
2,民國113年02月06日,交通,簡舒培
3,民國113年02月05日,工務,詹為元
4,民國113年02月05日,交通,詹為元
...,...,...,...
515,民國112年06月07日,交通,李明賢
516,民國112年06月07日,工務,李明賢
517,民國112年06月07日,工務,李明賢
518,民國112年06月07日,工務,李明賢


In [27]:
row_data = pd.read_csv("./table_29to107.csv", encoding="utf_8_sig", index_col=0)
print(row_data)     # 檢查
# type(row_data['Date'][0])   # 確認是string

             Date Category Name
0    民國113年02月06日       教育  張志豪
1    民國113年02月06日     財政建設  張志豪
2    民國113年02月06日       交通  簡舒培
3    民國113年02月05日       工務  詹為元
4    民國113年02月05日       交通  詹為元
..            ...      ...  ...
515  民國112年06月07日       交通  李明賢
516  民國112年06月07日       工務  李明賢
517  民國112年06月07日       工務  李明賢
518  民國112年06月07日       工務  李明賢
519  民國112年06月07日       交通  李明賢

[520 rows x 3 columns]


str

### translate time to AD for panda timesamp  

In [40]:
''' 將民國換成西元以便轉成panda timestamp '''

def turn_AD(date_str):
    date = re.findall(r'\d+',date_str)
    date[0] = str(int(date[0]) + 1911)
    date ='/'.join(date)
    return date

def turn_Timestamp(df):
    day = []        
    for i in range(len(df)):
        date_str = df["Date"][i]
        # print(date_str)
        day.append(turn_AD(date_str))
    # print(day)
    dayf = pd.DataFrame(day, columns=["Date"])
    df = df.copy()
    df['Date']= pd.to_datetime(dayf['Date'])    # 之後才能算時間區間
    return df
df = turn_Timestamp(row_data)

type(df['Date'][0])       #確認成功轉換
print(df)

          Date Category Name
0   2024-02-06       教育  張志豪
1   2024-02-06     財政建設  張志豪
2   2024-02-06       交通  簡舒培
3   2024-02-05       工務  詹為元
4   2024-02-05       交通  詹為元
..         ...      ...  ...
515 2023-06-07       交通  李明賢
516 2023-06-07       工務  李明賢
517 2023-06-07       工務  李明賢
518 2023-06-07       工務  李明賢
519 2023-06-07       交通  李明賢

[520 rows x 3 columns]


### 完成資料蒐集, 畫圖表分析
- 先確認有中文字在mambaforge裡
- 資料分析
    - 某時間區間的個人提案數 

In [41]:
''' 解決畫圖的中文輸入 '''
mpl.rcParams['font.sans-serif'] = ['Taipei Sans TC Beta']  # 指定中文字體
mpl.rcParams['axes.unicode_minus'] = False  # 解決負號'-'無法顯示的問題

In [ ]:
'''某時間區間的個人提案數'''
def date_filter(_df, date_s='2023-6-01', date_e='2024-02-30'):
    df = _df.copy()
    select = df.loc[(df['Date'] >= date_s) & (df['Date'] < date_e)]
    name_cnt = select["Name"].value_counts()
    print(name_cnt[:10])
    return name_cnt[:10]

print("2023整年個人質詢數目前10名")
# date_filter(df)

# print("2022前半年個人質詢數目前10名")
# date_filter(df,date_s="2021-12-30",date_e="2022-06-30")

# print("2022後半年個人質詢數目前10名")
# date_filter(df,date_s="2022-06-30",date_e="2022-12-30")

In [ ]:



'''正值選舉期間議員的書面質詢件數會減少嗎?'''
def bar_chart(df):
    x = df.index
    y = df
    plt.bar(x, y)
    plt.title('2022後半年書面質詢件數前十名', fontsize =16)
    plt.xlabel("姓名", fontsize =12)
    plt.ylabel("件數", fontsize =12)
    for i, v in enumerate(y):
        plt.text(i, v, str(v), ha='center')
    plt.savefig('halfyear_thebest10.png')
    plt.show()    

a = date_filter(df,date_s="2022-05-26",date_e="2022-11-26")
bar_chart(a.iloc[:10])

''' 以部門分類 '''

def filter_category_latest(_df, date_s='2022-06-01', date_e='2022-12-30', category="交通"):
    df = _df.copy()
    df_filtered = df.loc[(df['Date'] >= date_s) & (df['Date'] < date_e) & (df['Category']==category)]
    name_cnt = df_filtered["Name"].value_counts()
    print(f"2022後半年關於{category}部門書面質詢有: {len(df_filtered)} 筆")
    print(f"針對{category}部門書面質詢前十名的人如下：")
    print(name_cnt[:10])

cats = ["交通", "工務", "警政衛生", "民政", "教育", "財政建設"]
for c in cats:
    filter_category_latest(df, category= c)
    print("----------------------------------")


def filter_category_former(_df, date_s='2021-12-30', date_e='2022-06-30', category="交通"):
    df = _df.copy()
    df_filtered = df.loc[(df['Date'] >= date_s) & (df['Date'] < date_e) & (df['Category']==category)]
    name_cnt = df_filtered["Name"].value_counts()
    print(f"2022前半年關於{category}部門書面質詢有: {len(df_filtered)} 筆")
    print(f"針對{category}部門書面質詢前十名的人如下：")
    print(name_cnt[:10])

filter_category_former(df, category="交通")

 
def pie_chart(df,category:str,name:str):
    txt = df.index
    n = df
    plt.pie(n, labels=txt,autopct="%1.1f%%")
    plt.title(f'半年內{category}案件數量前十名')
    plt.savefig(f'{name}')
    plt.show()
   
def create_pie_chart(_df, category):
    df = _df.copy()
    df_filtered = df.loc[(df['Date'] >= '2022-06-30') & (df['Date'] < '2022-12-30') & (df['Category']==category)]
    name_cnt = df_filtered["Name"].value_counts()
    pie_chart(name_cnt.iloc[:10],category,f"halfyear_{category}")

print("")
for c in cats:
    create_pie_chart(df, c)
    

'''某時間區間提案少於五件的議員名單'''
a=[]
df_filtered = df.loc[(df['Date'] >= '2021-12-01') & (df['Date'] < '2022-12-01')]
name_cnt = df_filtered["Name"].value_counts()
print("2022整年書面質詢少於五件的議員名單：")
for n, v in zip(name_cnt.index, name_cnt):
    if v <= 5:
        print(n,v)
        a.append(v) #查看總共幾位
print(f"總共有: {len(a)} 位議員") 

for c in cats:
    df_filtered = df.loc[(df['Date'] >= '2021-12-01') & (df['Date'] < '2022-12-30') & (df['Category']==c)]
    res = df_filtered["Date"].dt.to_period('M').apply(lambda r: r.start_time)
    data = res.value_counts().sort_index()
    print(data)
    plt.plot(data.index,data, label=c)
    plt.legend()
    plt.show()



''' 時間區間的警政衛生項目每月幾筆 '''
for c in cats:
    df_filtered = df.loc[(df['Date'] >= '2021-12-01') & (df['Date'] < '2022-12-30') & (df['Category']==c)]
    res = df_filtered["Date"].dt.to_period('M').apply(lambda r: r.start_time)
    data = res.value_counts().sort_index()

    if c == "警政衛生":
        '''警政衛生的月份少了2022-12-01'''
        date = pd.to_datetime('2022-12-01')
        data = pd.concat([data, pd.Series([0], index=[date])])

    plt.bar(data.index,data, label=c,width=15)
plt.title('一年內的各項目每月件數 ')
plt.xlabel('月份')
plt.ylabel('件數')
plt.legend()
plt.savefig('all_item_yearcases.png')
plt.show()


In [ ]:
'''將時間換成西元以便轉成panda timestamp'''
# all = pd.read_csv("./final_project/table.csv", index_col=0)
# all["Date"]

def turn_AD(date_str):
    date = re.findall(r'\d+',date_str)
    date[0] = str(int(date[0]) + 1911)
    dates ='/'.join(date)
    return dates
# turn_AD(all["Date"][2])   #test
day = []        
for i in range(len(df)):
    date_str = df["Date"][i]
    # print(date_str)
    day.append(turn_AD(date_str))
    
# print(day)
  
dayf = pd.DataFrame(day, columns=["Date"])

df['Date']= pd.to_datetime(dayf['Date'])
# type(df['Date'][0])

In [ ]:
df['Date']

In [ ]:
df.info()

In [ ]:
a = df["Category"].value_counts()
a

In [ ]:
df.head(10)

In [ ]:
def browse(page:int,num:int):
    print(f"查看第{page}頁,第{num}筆:")
    data = (page-1)*10 + (num-1)
    print(df.loc[data])
browse(1,4)

In [ ]:
'''某時間區間的個人提案數'''
def date_filter(st:str,ed:str):    
    # print(all)
    select = df.loc[(df['Date'] >= st) & (df['Date'] < ed)]
    name_cnt = select["Name"].value_counts()
    return select,name_cnt

select, name_cnt = date_filter('2022-05-26','2022-11-26')
name_cnt[:10]

In [ ]:
mpl.rcParams['font.sans-serif'] = ['Taipei Sans TC Beta']  # 指定中文字體
mpl.rcParams['axes.unicode_minus'] = False  # 解決負號'-'無法顯示的問題

In [ ]:
'''正值選舉期間議員的書面質詢件數會減少嗎?'''
def bar_chart(df):
    x = df.index
    y = df
    plt.bar(x, y)
    plt.title('半年內書面質詢件數前十名', fontsize =16)
    plt.xlabel("姓名", fontsize =12)
    plt.ylabel("件數", fontsize =12)
    for i, v in enumerate(y):
        plt.text(i, v, str(v), ha='center')
    plt.savefig('halfyear_thebest10.png')
    plt.show()    
bar_chart(name_cnt.iloc[:10])

In [ ]:
'''某時間區間"交通"項目由誰提案最多(哪一區較常有交通糾紛)'''
df_filtered = df.loc[(df['Date'] >= '2021-12-01') & (df['Date'] < '2022-12-01') & (df['Category']=="交通")]
name_cnt_traffic = df_filtered["Name"].value_counts()
print(len(df_filtered))
print(name_cnt_traffic)
# df_filtered

In [ ]:
'''某時間區間"警政衛生"項目由誰提案最多(哪一區較常有警政衛生糾紛)'''
df_filtered = df.loc[(df['Date'] >= '2021-12-01') & (df['Date'] < '2022-12-01') & (df['Category']=="警政衛生")]
name_cnt_police = df_filtered["Name"].value_counts()
print(len(df_filtered))
print(name_cnt_police)
# df_filtered

In [ ]:
'''某時間區間"民政"項目由誰提案最多(哪一區較常有民政糾紛)'''
df_filtered = df.loc[(df['Date'] >= '2021-12-01') & (df['Date'] < '2022-12-01') & (df['Category']=="民政")]
name_cnt_civilaffairs = df_filtered["Name"].value_counts()
print(len(df_filtered))
print(name_cnt_civilaffairs)

In [ ]:
'''某時間區間"教育"項目由誰提案最多(哪一區較常有教育糾紛)'''
df_filtered = df.loc[(df['Date'] >= '2021-12-01') & (df['Date'] < '2022-12-01') & (df['Category']=="教育")]
name_cnt_education = df_filtered["Name"].value_counts()
print(len(df_filtered))
print(name_cnt_education)

In [ ]:
def pie_chart(df,category:str,name:str):
    txt = df.index
    n = df
    plt.pie(n, labels=txt,autopct="%1.1f%%")
    plt.title(f'半年內{category}案件數量前十名')
    plt.savefig(f'{name}')
    plt.show()


In [ ]:
pie_chart(name_cnt_traffic.iloc[:10],'交通',"halfyear_traffics")
pie_chart(name_cnt_police.iloc[:10],'警政衛生',"halfyear_PoliceHygiene")
pie_chart(name_cnt_civilaffairs.iloc[:10],'民政',"halfyear_civilaffairs")
pie_chart(name_cnt_education.iloc[:10],'教育',"halfyear_education")



In [ ]:
'''某時間區間提案少於五件的議員名單'''
a=[]
df_filtered = df.loc[(df['Date'] >= '2021-12-01') & (df['Date'] < '2022-12-01')]
name_cnt = df_filtered["Name"].value_counts()
print("2022整年書面質詢少於五件的議員名單：")
for n, v in zip(name_cnt.index, name_cnt):
    if v <= 5:
        # print(f"Get out {n} :), w/ {v} issues.")
        print(n,v)
        a.append(v) #查看總共幾位
print(f"總共有: {len(a)} 位議員")


In [ ]:
''' 各項目提案的時間 '''
df_ls = df.groupby('Category')['Date'].apply(list)
df_ls

In [ ]:
''' 時間區間的交通項目每月幾筆 '''
# df_ls.index
# df_ls[0]
df_filtered = df.loc[(df['Date'] >= '2021-12-01') & (df['Date'] < '2022-12-28') & (df['Category']=="交通")]
res = df_filtered["Date"].dt.to_period('M').apply(lambda r: r.start_time)
traf = res.value_counts().sort_index()

print(traf)
plt.plot(traf.index,traf, label='交通',color='orange')
plt.legend()
plt.show()

In [ ]:
''' 時間區間的警政衛生項目每月幾筆 '''
# df_ls.index
# df_ls[0]
df_filtered = df.loc[(df['Date'] >= '2021-12-01') & (df['Date'] < '2022-12-28') & (df['Category']=="警政衛生")]
res = df_filtered["Date"].dt.to_period('M').apply(lambda r: r.start_time)
police = res.value_counts().sort_index()


print(police)
plt.plot(police.index,police, label='警政衛生',color='salmon')
plt.legend()
plt.show()

In [ ]:
''' 時間區間的財政建設項目每月幾筆 '''
# df_ls.index
# df_ls[0]
df_filtered = df.loc[(df['Date'] >= '2021-12-01') & (df['Date'] < '2022-12-28') & (df['Category']=="財政建設")]
res = df_filtered["Date"].dt.to_period('M').apply(lambda r: r.start_time)
finance = res.value_counts().sort_index()
print(finance)
plt.plot(finance.index,finance, label='財政建設',color='r')
plt.legend()
plt.show()

In [ ]:
''' 時間區間的工務項目每月幾筆 '''
# df_ls.index
# df_ls[0]
df_filtered = df.loc[(df['Date'] >= '2021-12-01') & (df['Date'] < '2022-12-28') & (df['Category']=="工務")]
res = df_filtered["Date"].dt.to_period('M').apply(lambda r: r.start_time)
public_work = res.value_counts().sort_index()
print(public_work)
plt.plot(public_work.index,public_work, label='工務',color='plum')
plt.legend()
plt.show()

In [ ]:
''' 時間區間的教育項目每月幾筆 '''
# df_ls.index
# df_ls[0]
df_filtered = df.loc[(df['Date'] >= '2021-12-01') & (df['Date'] < '2022-12-28') & (df['Category']=="教育")]
res = df_filtered["Date"].dt.to_period('M').apply(lambda r: r.start_time)
educate = res.value_counts().sort_index()
print(educate)
plt.plot(educate.index,educate, label='教育',color='darkseagreen')
plt.legend()
plt.show()

In [ ]:
''' 時間區間的民政項目每月幾筆 '''
# df_ls.index
# df_ls[0]
df_filtered = df.loc[(df['Date'] >= '2021-12-01') & (df['Date'] < '2022-12-28') & (df['Category']=="民政")]
res = df_filtered["Date"].dt.to_period('M').apply(lambda r: r.start_time)
civil = res.value_counts().sort_index()
print(civil)

plt.plot(civil.index,civil, label='民政',color='skyblue')
plt.legend()
plt.show()

In [ ]:
'''警政衛生的月份少了2022-12-01'''
date = pd.to_datetime('2022-12-01')
new_row = pd.Series([0], index=[date])
newpolice = pd.concat([police,new_row])
newpolice

In [ ]:
plt.bar(newpolice.index,newpolice, label='警政衛生',width=15)
plt.bar(educate.index,educate, label='教育',width=15)
plt.bar(traf.index,traf, label='交通',width=15)
plt.bar(public_work.index,public_work, label='工務',width=15)
plt.bar(finance.index,finance, label='財政建設',width=15)
plt.bar(civil.index,civil, label='民政',width=15)

plt.title('一年內的各項目每月件數 ')
plt.xlabel('月份')
plt.ylabel('件數')
plt.legend()
plt.savefig('all_item_yearcases.png')
plt.show()


In [ ]:
name_cnt = df["Name"].value_counts()
date_cnt = df["Date"].value_counts()
cate_cnt = df["Category"].value_counts()
print(name_cnt,date_cnt,cate_cnt)

In [ ]:
''' 將中文字導入matplotlib.font '''
# import matplotlib
# print(matplotlib.__file__)
''' 刪掉既有的快取文字字體(找fontList.json並刪除) '''
# import matplotlib
# print(matplotlib.get_cachedir())
''' 確認字體 ''' 
import matplotlib
import matplotlib.font_manager
a = sorted([f.name for f in matplotlib.font_manager.fontManager.ttflist])
 
for i in a:
    print(i)

In [ ]:
import re
import time
import numpy as np
import pandas as pd
import requests as rq
from bs4 import BeautifulSoup
import matplotlib.pyplot as plt
import matplotlib as mpl 
import final_project.utils as u

In [ ]:
df = pd.read_csv("./table_1to78.csv", encoding="utf_8_sig", index_col=0)
df = u.turn_Timestamp(df)

In [ ]:
cats = ["交通", "工務", "警政衛生", "民政", "教育", "財政建設"]
"""折線圖"""
for c in cats:
    df_filtered = df.loc[(df['Date'] >= '2021-12-01') & (df['Date'] < '2022-12-30') & (df['Category']==c)]
    res = df_filtered["Date"].dt.to_period('M').apply(lambda r: r.start_time)
    data = res.value_counts().sort_index()
    print(data)
    print(data.index)
    
    plt.plot(data.index, data, label=c)
    plt.legend()
    plt.show()

In [ ]:
[str(i) for i in data.index]

In [ ]:
data.index

In [ ]:
[i.strftime("%m") for i in data.index]